# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 
import re

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt


def clean_non_western_encoded_characters_commands(text: str) -> str:
    """ Remove non-western encoded characters from a string
    List may need to grow.
    
    :param text: the text to clean
    :return: the cleaned text
    """
    text = re.sub(r"(\\begin{CJK}{UTF8}{gbsn})(.*?)(\\end{CJK})", r"\2", text)
    return text


def get_initials(name: str) -> str:
    """ Get the short name, e.g., A.-B. FamName
    :param name: full name
    :returns: initials
    """
    initials = []
    # account for non western names often in ()
    if '(' in name:
        name = clean_non_western_encoded_characters_commands(name)
        suffix = re.findall(r"\((.*?)\)", name)[0]
        name = name.replace(f"({suffix})", '')
    else:
        suffix = ''
    split = name.split()
    for token in split[:-1]:
        if '-' in token:
            current = '-'.join([k[0] + '.' for k in token.split('-')])
        else:
            current = token[0] + '.'
        initials.append(current)
    initials.append(split[-1].strip())
    if suffix:
        initials.append(f"({suffix})")
    return ' '.join(initials)

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# deal with the author list and edge cases of people that cannot be consistent on their name  

def filter_non_scientists(name: str) -> bool:
    """ Loose filter on expected authorships

    removing IT, administration, technical staff
    :param name: name
    :returns: False if name is not a scientist
    """
    remove_list = ['Licht', 'Binroth', 'Witzel', 'Jordan',
                   'Zähringer', 'Scheerer', 'Hoffmann', 'Düe',
                   'Hellmich', 'Enkler-Scharpegge', 'Witte-Nguy',
                   'Dehen', 'Beckmann', 'Jager', 'Jäger'
                  ]

    for k in remove_list:
        if k in name:
            return False
    return True

def add_author_to_list(author_list: list) -> list:
    """ Add author to list if not already in list
    
    :param author: author name
    :param author_list: list of authors
    :returns: updated list of authors
    """
    add_list = ['T. Henning']

    for author in add_list:
        if author not in author_list:
            author_list.append(author)
    return author_list

# get list from MPIA website
# filter for non-scientists (mpia.get_mpia_mitarbeiter_list() does some filtering)
mpia_authors = [k[1] for k in mpia.get_mpia_mitarbeiter_list() if filter_non_scientists(k[1])]
# add some missing author because of inconsistencies in their MPIA name and author name on papers
mpia_authors = add_author_to_list(mpia_authors)

In [4]:
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

def robust_call(fn, value, *args, **kwargs):
    try:
        return fn(value, *args, **kwargs)
    except Exception:
        return value

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [robust_call(mpia.get_initials, k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

X. Zhang  ->  X. Zhang  |  ['X. Zhang']
E. Bañados  ->  E. Bañados  |  ['E. Bañados']
Y. Khusanova  ->  Y. Khusanova  |  ['Y. Khusanova']
F. Walter  ->  F. Walter  |  ['F. Walter']
S. Barboza  ->  S. Barboza  |  ['S. Barboza']
M. Sharma  ->  Y. M. Sharma  |  ['M. Sharma']
A. d. Graaff  ->  A. D. Graaff  |  ['A. D. Graaff']
F. Walter  ->  F. Walter  |  ['F. Walter']


T. Henning  ->  T. Henning  |  ['T. Henning']
Arxiv has 72 new papers today
          7 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates):
    # debug crap
    paper['identifier'] = paper['identifier'].lower().replace('arxiv:', '').replace(r'\n', '').strip()
    paper_id = paper['identifier']
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [get_initials(k) for k in doc.authors], 
                mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print("Issues with the citations")
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/7 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2412.06873
extracting tarball to tmp_2412.06873...

 done.
Retrieving document from  https://arxiv.org/e-print/2412.06894


extracting tarball to tmp_2412.06894...

 done.


...

Found 133 bibliographic references in tmp_2412.07474/J0305m3150_140pc.bbl.
Error retrieving bib data for Venemans2016: 'author'
Error retrieving bib data for Venemans2019: 'author'
Retrieving document from  https://arxiv.org/e-print/2412.07691


extracting tarball to tmp_2412.07691...

 done.


T. Henning  ->  T. Henning  |  ['T. Henning']


Found 188 bibliographic references in tmp_2412.07691/arxive.bbl.
Issues with the citations
syntax error in line 39: '=' expected


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2412.06894-b31b1b.svg)](https://arxiv.org/abs/2412.06894) | **A SPectroscopic survey of biased halos In the Reionization Era (ASPIRE): Spectroscopically Complete Census of Obscured Cosmic Star Formation Rate Density at $z=4-6$**  |
|| F. Sun, et al. -- incl., <mark>E. Bañados</mark>, <mark>Y. Khusanova</mark>, <mark>F. Walter</mark> |
|*Appeared on*| *2024-12-11*|
|*Comments*| *27 pages, 13 figures, 3 tables. Accepted for publication in ApJ*|
|**Abstract**|            We present a stringent measurement of the dust-obscured star-formation rate density (SFRD) at $z=4-6$ from the ASPIRE JWST Cycle-1 medium and ALMA Cycle-9 large program. We obtained JWST/NIRCam grism spectroscopy and ALMA 1.2-mm continuum map along 25 independent quasar sightlines, covering a total survey area of $\sim$35 arcmin$^2$ where we search for dusty star-forming galaxies (DSFGs) at $z = 0 - 7$. We identify eight DSFGs in seven fields at $z=4-6$ through the detection of H$\alpha$ or [O III] $\lambda$5008 lines, including fainter lines such as H$\beta$, [O III] $\lambda$4960, [N II] $\lambda$6585, [S II] $\lambda\lambda$6718,6733 for six sources. With this spectroscopically complete DSFG sample at $z=4-6$ and negligible impact from cosmic variance (shot noise), we measure the infrared luminosity function (IRLF) down to $L_\mathrm{IR} \sim 2\times10^{11}$ $L_\odot$. We find flattening of IRLF at $z=4-6$ towards the faint end (power-law slope $\alpha = 0.59_{-0.45}^{+0.39}$). We determine the dust-obscured cosmic SFRD at this epoch as $\log[\rho_\mathrm{SFR,IR} / (\mathrm{M}_\odot\,\mathrm{yr}^{-1}\,\mathrm{Mpc}^{-3})] = -1.52_{-0.13}^{+0.14}$. This is significantly higher than previous determination using ALMA data in the Hubble Ultra Deep Field, which is void of DSFGs at $z=4-6$ because of strong cosmic variance (shot noise). We conclude that the majority ($66\pm7$%) of cosmic star formation at $z \sim 5$ is still obscured by dust. We also discuss the uncertainty of SFRD propagated from far-IR spectral energy distribution and IRLF at the bright end, which will need to be resolved with future ALMA and JWST observations.         |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2412.06897-b31b1b.svg)](https://arxiv.org/abs/2412.06897) | **Little iLocater: paving the way for iLocater**  |
|| R. J. Harris, et al. -- incl., <mark>S. Barboza</mark> |
|*Appeared on*| *2024-12-11*|
|*Comments*| *13 pages, 17 figures, accepted for publication in MNRAS*|
|**Abstract**|            Diffraction-limited radial velocity instruments offer a pathway towards improved precision and stability, and the exploration of new parameter spaces at high spatial and spectral resolution. However, achieving the necessary performance requires careful instrument design and considerable on-sky testing. We describe the design and construction of ``Little iLocater'' (Lili), a compact spectrograph that has been used to validate the performance of the front-end fibre-injection system of the iLocater spectrograph. We present the design, assembly, and performance using on-sky data obtained at the Large Binocular Telescope (LBT), including extraction of spectra from standard stars, testing of the atmospheric dispersion corrector to elevations of 40 degrees, and spatially resolved spectra from close companion systems. These results show the front-end fibre-injection system is performing as expected and is indicative of iLocater's capabilities once installed at the LBT.         |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2412.07474-b31b1b.svg)](https://arxiv.org/abs/2412.07474) | **Pushing ALMA to the limit: 140 pc resolution observations of a z=6.6 quasar-galaxy merger resolve strikingly different morphologies of dust continuum and [CII] 158 um emission**  |
|| R. A. Meyer, et al. -- incl., <mark>F. Walter</mark> |
|*Appeared on*| *2024-12-11*|
|*Comments*| *Accepted in ApJ. 15 pages + appendices*|
|**Abstract**|            We present $0."026$ $(140\ \rm{pc})$ resolution ALMA observations of [C II] $158\ \mu\rm{m}$ and dust continuum emission of the $z=6.6$ quasar J0305--3150, resolved over $\sim 300-400$ independent resolution elements. The dust continuum emission is compact with $\sim 80\%$ recovered within $r<0."3$ $(1.6\ \rm{kpc})$, whereas the [C II] emission profile is composed of a central Gaussian ($r<0."4$, i.e. $<2.2\ \rm{kpc}$) and an extended component (detected up to $\sim 10\ \rm{kpc}$ at $>3\sigma$). We infer a direct contribution of the quasar to the observed 260\ \rm{GHz} continuum $S_{\nu,\rm{QSO}} / S_{\nu,\rm{QSO+Host}} \lesssim 1\%$. We report the detection of FIR-detected star-forming clumps with $r<200 \ \rm{pc}$ and properties similar to that of rest-frame UV-optical clumps reported in the literature. The $200\ \rm{pc}$ resolved [C II]/FIR ratio follows the global relation with the FIR surface brightness established in low- and high-redshift galaxies, even at the quasar location. We find that dust continuum is emitted in regions of $\sim0."02-0."04$ consistent with the size of photo-dissociation regions (PDR), whereas $50\%$ of the [C II] originates from larger physical scales ($\theta \gtrsim 2"$). The large-scale [C II] emission presents a velocity gradient aligned with a nearby companion with perturbed kinematics, and misaligned with the kinematics of the small-scale emission. The absence of significant [C II] emission by structures with physical scale $\lesssim 1\ \rm{kpc}$ implies that [C II] emission is not produced in dense PDR located at the boundary of Giant Molecular Clouds. We argue instead that [C II] is produced in low-density PDRs in the interstellar medium and diffuse HI gas tidally-stripped during the ongoing merger.         |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2412.07691-b31b1b.svg)](https://arxiv.org/abs/2412.07691) | **The CARMENES search for exoplanets around M dwarfs. The impact of rotation and magnetic fields on the radial velocity jitter in cool stars**  |
|| H. L. Ruh, et al. -- incl., <mark>T. Henning</mark> |
|*Appeared on*| *2024-12-11*|
|*Comments*| **|
|**Abstract**|            Radial velocity (RV) jitter represents an intrinsic limitation on the precision of Doppler searches for exoplanets that can originate from both instrumental and astrophysical sources. We aim to determine the RV jitter floor in M dwarfs and investigate the stellar properties that lead to RV jitter induced by stellar activity. We determined the RV jitter in 239 M dwarfs from the CARMENES survey that are predominantly of mid to late spectral type and solar metallicity. We also investigated the correlation between stellar rotation and magnetic fields with RV jitter. The median jitter in the CARMENES sample is 3.1 m/s, and it is 2.3 m/s for stars with an upper limit of 2 km/s on their projected rotation velocities. We provide a relation between the stellar equatorial rotation velocity and RV jitter in M dwarfs based on a subsample of 129 well-characterized CARMENES stars. RV jitter induced by stellar rotation dominates for stars with equatorial rotation velocities greater than 1 km/s. A jitter floor of 2 m/s dominates in stars with equatorial rotation velocities below 1 km/s. This jitter floor likely contains contributions from stellar jitter, instrumental jitter, and undetected companions. We study the impact of the average magnetic field and the distributions of magnetic filling factors on the RV jitter. We find a series of stars with excess RV jitter and distinctive distributions of magnetic filling factors. These stars are characterized by a dominant magnetic field component between 2-4 kG. An RV jitter floor can be distinguished from RV jitter induced by activity and rotation based on the stellar equatorial rotation velocity. RV jitter induced by activity and rotation primarily depends on the equatorial rotation velocity. This RV jitter is also related to the distribution of magnetic filling factors, and this emphasizes the role of the magnetic field in the generation of RV jitter.         |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2412.06873-b31b1b.svg)](https://arxiv.org/abs/2412.06873) | **Revisiting holographic dark energy from the perspective of multi-messenger gravitational wave astronomy: future joint observations with short gamma-ray bursts**  |
|| T. Han, Z. Li, J.-F. Zhang, <mark>X. Zhang</mark> |
|*Appeared on*| *2024-12-11*|
|*Comments*| *13 pages, 8 figures*|
|**Abstract**|            The advent of third-generation (3G) gravitational-wave (GW) detectors opens new opportunities for multi-messenger observations of binary neutron star merger events, holding significant potential for probing the history of cosmic expansion. In this paper, we investigate the holographic dark energy (HDE) model by using the future GW standard siren data observed from the 3G GW detectors and the short $\gamma$-ray burst THESEUS-like detector joint observations. We find that GW data alone can achieve a relatively precise estimation of the Hubble constant, with precision of $0.2\%$-$0.6\%$, but its ability to constrain other cosmological parameters remains limited. Nonetheless, since the GW data can break parameter degeneracies generated by the mainstream EM observations, CMB + BAO + SN (CBS), GW standard sirens play a crucial role in enhancing the accuracy of parameter estimation. With the addition of GW data to CBS, the constraints on cosmological parameters $H_0$, $c$ and $\Omega_{\rm{m}}$ can be improved by $63\%$-$88\%$, $27\%$-$44\%$ and $55\%$-$70\%$. In summary, observations of GW standard sirens from 3G GW detectors could be pivotal in helping solve the Hubble tension and probe the fundamental nature of dark energy.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2412.06929-b31b1b.svg)](https://arxiv.org/abs/2412.06929) | **Physical characterization of the FeLoBAL outflow in SDSS J0932+0840: Analysis of VLT/UVES observations**  |
|| <mark>M. Sharma</mark>, et al. |
|*Appeared on*| *2024-12-11*|
|*Comments*| *15 pages and 10 figures. Accepted for publication in A&A*|
|**Abstract**|            Context: The study of quasar outflows is essential in understanding the connection between active galactic nuclei (AGN) and their host galaxies. We analyze the VLT/UVES spectrum of quasar SDSS J0932+0840 and identify several narrow and broad outflow components in absorption, with multiple ionization species including Fe II, which puts it among a rare class of outflows known as FeLoBALs. Aims: We study one of the outflow components to determine its physical characteristics by determining the total hydrogen column density, ionization parameter and the hydrogen number density. Through these parameters, we aim to obtain the distance of the outflow from the central source, its mass outflow rate and kinetic luminosity, and to constrain the contribution of the outflow to AGN feedback. Methods: We obtain the ionic column densities from the absorption troughs in the spectrum, and use photoionization modeling to extract the physical parameters of the outflow, including the total hydrogen column density and ionization parameter. The relative population of the observed excited states of Fe II is used to model the hydrogen number density of the outflow. Results: We use the Fe II excited states to model the electron number density ($n_e$) and hydrogen number density ($n_H$) independently and obtain $n_e$ $\simeq$ $10^{3.4}$ cm$^{-3}$ and $n_H$ $\simeq$ $10^{4.8}$ cm$^{-3}$. Our analysis of the physical structure of the cloud shows that these two results are consistent with each other. This places the outflow system at a distance of $0.7_{-0.4}^{+0.9}$ kpc from the central source, with mass flow rate ($\dot{M}$) of $43^{+65}_{-26}$ $M_\odot$ yr$^{-1}$ and kinetic luminosity ($\dot{E_k}$) of $0.7^{+1.1}_{-0.4}$ $\times$ $10^{43}$ erg s$^{-1}$.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2412.06957-b31b1b.svg)](https://arxiv.org/abs/2412.06957) | **JWST UNCOVERs the Optical Size - Stellar Mass Relation at $4<z<8$: Rapid Growth in the Sizes of Low Mass Galaxies in the First Billion Years of the Universe**  |
|| T. B. Miller, et al. -- incl., <mark>A. d. Graaff</mark> |
|*Appeared on*| *2024-12-11*|
|*Comments*| *Submitted to ApJ, comments welcome!*|
|**Abstract**|            We study the rest-frame optical and ultraviolet morphology of galaxies in the first billion years of the Universe. Using JWST data from the UNCOVER and MegaScience surveys targeting the lensing cluster Abell 2744 we present multi-band morphological measurements for a sample of 995 galaxies selected using 20-band NIRCam photometry and 35 using NIRSpec Prism spectroscopy over the redshift range of $4<z<8$. The wavelength-dependent morphology is measured using pysersic by simultaneously modeling the images in 6 NIRCam wide filters covering the rest-frame UV to optical. The joint modeling technique increases the precision of measured radii by 50\%. Galaxies in our sample show a wide range of Sersic indices, with no systematic difference between optical and UV morphology. We model the size-mass relation in a Bayesian manner using a continuity model to directly fit the redshift evolution while accounting for observational uncertainties. We find the average size of galaxies at $\log M_*/M_\odot=8.5$ grows rapidly, from 400 pc at $z=8$ to 830 pc at $z=4$. This is faster evolution than expected from power law scalings of the Hubble parameter or scale factor that describe well previous results at $z<2$. This suggests that different and/or much stronger processes affect low mass systems during the epoch of reionization. The measured logarithmic slope (0.25) and scatter (0.23 dex) are non-evolving. We discuss the remarkable consistency of the slope and scatter over cosmic time in the context of the galaxy-halo connection.         |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error bad escape \i at position 38</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    print("found figures", fig_fnames)
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        if not os.path.exists(fname):
            print("file not found", fname)
            continue
        print("copying ", fname, "to", directory)
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

found figures ['tmp_2412.06894/./figure/all_alma_field.png', 'tmp_2412.06894/./figure/completeness_simulation.png', 'tmp_2412.06894/./figure/cycle_11_fig_1_jwst_alma_cutout.png']
copying  tmp_2412.06894/./figure/all_alma_field.png to _build/html/
copying  tmp_2412.06894/./figure/completeness_simulation.png to _build/html/
copying  tmp_2412.06894/./figure/cycle_11_fig_1_jwst_alma_cutout.png to _build/html/
exported in  _build/html/2412.06894.md
    + _build/html/tmp_2412.06894/./figure/all_alma_field.png
    + _build/html/tmp_2412.06894/./figure/completeness_simulation.png
    + _build/html/tmp_2412.06894/./figure/cycle_11_fig_1_jwst_alma_cutout.png
found figures ['tmp_2412.06897/./images/zemax_3d.png', 'tmp_2412.06897/./images/lili_transmission.png', 'tmp_2412.06897/./images/lili_footprint.png']
copying  tmp_2412.06897/./images/zemax_3d.png to _build/html/
copying  tmp_2412.06897/./images/lili_transmission.png to _build/html/
copying  tmp_2412.06897/./images/lili_footprint.png to _buil

## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\vdag}{(v)^\dagger}$
$\newcommand$
$\newcommand$
$\newcommand{\oii}{\mbox{[\ion{O}{2}]}}$
$\newcommand{\oiii}{\mbox{[\ion{O}{3}]}}$
$\newcommand{\hei}{\mbox{\ion{He}{1}}}$
$\newcommand{\heii}{\mbox{\ion{He}{2}}}$
$\newcommand{\cii}{\mbox{[\ion{C}{2}]}}$
$\newcommand{\nii}{\mbox{[\ion{N}{2}]}}$
$\newcommand{\sii}{\mbox{[\ion{S}{2}]}}$
$\newcommand{\siii}{\mbox{[\ion{S}{3}]}}$
$\newcommand{\hb}{\mbox{H\beta}}$
$\newcommand{\ha}{\mbox{H\alpha}}$
$\newcommand{\lya}{\mbox{Ly\alpha}}$
$\newcommand{\popiii}{\mbox{\ion{Pop}{3}}}$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand{\zsp}{\mbox{z_\mathrm{spec}}}$
$\newcommand{\zph}{\mbox{z_\mathrm{phot}}}$
$\newcommand{\lir}{\mbox{L_\mathrm{IR}}}$
$\newcommand{\textred}[1]{\textcolor{red}{#1}}$
$\newcommand{\textblue}[1]{\textcolor{blue}{#1}}$
$\newcommand{\micron}{\si{\micro\meter}}$
$\newcommand{\textred}[1]{{#1}}$
$\newcommand{\textblue}[1]{{#1}}$
$\newcommand{\thefigure}{\thesection\arabic{figure}}$</div>



<div id="title">

# A SPectroscopic survey of biased halos In the Reionization Era (ASPIRE): \ Spectroscopically Complete Census of Obscured Cosmic Star Formation Rate Density at $z=4-6$

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2412.06894-b31b1b.svg)](https://arxiv.org/abs/2412.06894)<mark>Appeared on: 2024-12-11</mark> -  _27 pages, 13 figures, 3 tables. Accepted for publication in ApJ_

</div>
<div id="authors">

F. Sun, et al. -- incl., <mark>Y. Khusanova</mark>, <mark>F. Walter</mark>

</div>
<div id="abstract">

**Abstract:** We present a stringent measurement of the dust-obscured star-formation rate density (SFRD) at $z=4-6$ from the ASPIRE JWST Cycle-1 medium and ALMA Cycle-9 large program.We obtained JWST/NIRCam grism spectroscopy and ALMA 1.2-mm continuum map along 25 independent quasar sightlines, covering a total survey area of $\sim$ 35 arcmin $^2$ where we search for dusty star-forming galaxies (DSFGs) at $z = 0 - 7$ .We identify eight DSFGs in seven fields at $z=4-6$ through the detection of $\ha$ or $\oiii$ $\lambda$ 5008 lines, including fainter lines such as $\hb$ , $\oiii$ $\lambda$ 4960, $\nii$ $\lambda$ 6585, $\sii$ $\lambda\lambda$ 6718,6733 for six sources.With this spectroscopically complete DSFG sample at $z=4-6$ and negligible impact from cosmic variance (shot noise), we measure the infrared luminosity function (IRLF) down to $L_\mathrm{IR} \sim 2\times10^{11}$ $\lsun$ .We find flattening of IRLF at $z=4-6$ towards the faint end (power-law slope $\alpha = 0.59_{-0.45}^{+0.39}$ ).We determine the dust-obscured cosmic SFRD at this epoch as $\log[\rho_\mathrm{SFR,IR} / (\mathrm{M}_\odot \mathrm{yr}^{-1} \mathrm{Mpc}^{-3})] = -1.52_{-0.13}^{+0.14}$ .This is significantly higher than previous determination using ALMA data in the Hubble Ultra Deep Field, which is void of DSFGs at $z=4-6$ because of strong cosmic variance (shot noise).We conclude that the majority ( $66\pm7$ \% ) of cosmic star formation at $z \sim 5$ is still obscured by dust.We also discuss the uncertainty of SFRD propagated from far-IR spectral energy distribution and IRLF at the bright end, which will need to be resolved with future ALMA and JWST observations.

</div>

<div id="div_fig1">

<img src="tmp_2412.06894/./figure/all_alma_field.png" alt="Fig4" width="100%"/>

**Figure 4. -** 1.2-mm continuum images of 25 quasar fields obtained by ASPIRE ALMA Cycle-9 large program. In the top-left panel, we highlight the design of JWST/NIRCam and ALMA observations.
The whole ALMA 1.2-mm continuum imaging mosaics (_uv_-tapered with FWHM = 1$\arcsec$) are within the full spectral ($\lambda$ = 3.1--4.0 $\micron$ with F356W filter) coverage region of NIRCam module A as indicated by the blue shaded region.
The quasar J0109--3047 ($z=6.791$; cyan circle) is located in the center of ALMA footprint, and one DSFG (J0109m3047.C02 at $z=5.549$) is also highlighted with red circle.
The position angle of NIRCam WFSS observation is 270\arcdeg for J0109 field, and therefore the grism-R dispersion direction is almost from north to south as indicated by the orange arrow.
Note that the dispersion direction depends on the JWST/NIRCam PA and varies from field to field.
ALMA 1.2-mm continuum images of all the other 24 fields are also displayed. Quasars with continuum detection ($z=6.5-6.8$) and spectroscopically confirmed DSFGs at $z=4-6$ are highlighted in cyan and red circles, respectively.
Many DSFGs at other redshifts are also detected with ALMA but not highlighted in this figure.
 (*fig:all_alma*)

</div>
<div id="div_fig2">

<img src="tmp_2412.06894/./figure/completeness_simulation.png" alt="Fig3" width="100%"/>

**Figure 3. -** Completeness as a function of ALMA flux density on 1$\farcs$0 _uv_-tapered images (before primary beam response correction).
Best-fit error function is shown as the solid black line.
 (*fig:complete*)

</div>
<div id="div_fig3">

<img src="tmp_2412.06894/./figure/cycle_11_fig_1_jwst_alma_cutout.png" alt="Fig5" width="100%"/>

**Figure 5. -** JWST NIRCam (red: F356W; green: F200W, blue: F115W) and ALMA 1.2-mm continuum images of DSFGs at $z=4-6$ discovered with the ASPIRE survey. Image sizes are 4$\arcsec$$\times$4$\arcsec$(north up, east left).
Source ID, spectroscopic redshifts and ALMA beam sizes are indicated in the plots.
Most sources appear red in JWST RGB images, indicating that they are highly dust-obscured galaxies at high redshifts.
Note that J0109m3047.C02 is gravitationally lensed by the bright galaxy on the left (see Appendix \ref{apd:01_lens}).
 (*fig:cutout*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2412.06894"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\thebibliography}{\DeclareRobustCommand{\VAN}[3]{##3}\VANthebibliography}$</div>



<div id="title">

# Little iLocater: paving the way for iLocater

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2412.06897-b31b1b.svg)](https://arxiv.org/abs/2412.06897)<mark>Appeared on: 2024-12-11</mark> -  _13 pages, 17 figures, accepted for publication in MNRAS_

</div>
<div id="authors">

R. J. Harris, et al. -- incl., <mark>S. Barboza</mark>

</div>
<div id="abstract">

**Abstract:** Diffraction-limited radial velocity instruments offer a pathway towards improved precision and stability, and the exploration of new parameter spaces at high spatial and spectral resolution. However, achieving the necessary performance requires careful instrument design and considerable on-sky testing. We describe the design and construction of "Little iLocater" (Lili), a compact spectrograph that has been used to validate the performance of the front-end fibre-injection system of the iLocater spectrograph. We present the design, assembly, and performance using on-sky data obtained at the Large Binocular Telescope (LBT), including extraction of spectra from standard stars, testing of the atmospheric dispersion corrector to elevations of $\qty{40}{◦ee}$ , and spatially resolved spectra from close companion systems. These results show the front-end fibre-injection system is performing as expected and is indicative of iLocater's capabilities once installed at the LBT.

</div>

<div id="div_fig1">

<img src="tmp_2412.06897/./images/zemax_3d.png" alt="Fig1" width="100%"/>

**Figure 1. -** Shaded rendering of the optical design of the \ac{Lili} spectrograph, with the light path coloured by wavelength. From left to right, the light from the \ac{SMF}  is collimated by an \ac{OAP}, which feeds a \ac{VPH} grating. Light is then refocused onto the detector by a TTL200-S8 and AC-508-080-C lens. (*fig:Lili_optical_design*)

</div>
<div id="div_fig2">

<img src="tmp_2412.06897/./images/lili_transmission.png" alt="Fig5" width="100%"/>

**Figure 5. -** Estimated spectrograph throughput based on commercially available values for lenses and theoretical values for the \ac{MCIFU} gratings. Individual efficiencies of the two spectrograph orders are plotted as dotted lines. The filled area underneath indicates the parts of the spectrum from each order incident on the \ac{CRED2} detector. (*fig:Lili_transmission*)

</div>
<div id="div_fig3">

<img src="tmp_2412.06897/./images/lili_footprint.png" alt="Fig3" width="100%"/>

**Figure 3. -** Footprint of the two orders imaged onto the detector. The first order (\qtyrange{0.97}{1.2}{\micro\meter}) crosses the full detector area diagonally, while the second sits below (\qtyrange{1.2}{1.34}{\micro\meter}). Note that there is a difference in slope between the first and second orders due to the rotation of the diffraction gratings. (*fig:Lili_footprint*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2412.06897"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\vdag}{(v)^\dagger}$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand\cii{[C\thinspace{\scriptsize II}]}$</div>



<div id="title">

# Pushing ALMA to the limit: 140 pc resolution observations of a z=6.6 quasar-galaxy merger resolve strikingly different morphologies of dust continuum and [$\ion{C}{2}$] 158$\mu\rm{m}$ emission

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2412.07474-b31b1b.svg)](https://arxiv.org/abs/2412.07474)<mark>Appeared on: 2024-12-11</mark> -  _Accepted in ApJ. 15 pages + appendices_

</div>
<div id="authors">

R. A. Meyer, et al. -- incl., <mark>F. Walter</mark>

</div>
<div id="abstract">

**Abstract:** We present $0\farcs026$ $(140 \rm{pc})$ resolution ALMA observations of [ $\ion{C}{2}$ ] $158 \mu\rm{m}$ and dust continuum emission of the $z=6.6$ quasar J0305--3150, resolved over $\sim 300-400$ independent resolution elements. The dust continuum emission is compact with $\sim 80\%$ recovered within $r<0\farcs3$ $(1.6 \rm{kpc})$ , whereas the [ $\ion{C}{2}$ ] emission profile is composed of a central Gaussian ( $r<0\farcs4$ , i.e. $<2.2 \rm{kpc}$ ) and an extended component (detected up to $\sim 10 \rm{kpc}$ at $>3\sigma$ ). We infer a direct contribution of the quasar to the observed 260 $\rm{GHz}$ continuum $S_{\nu,\rm{QSO}} / S_{\nu,\rm{QSO+Host}} \lesssim 1\%$ . We report the detection of FIR-detected star-forming clumps with $r<200  \rm{pc}$ and properties similar to that of rest-frame UV-optical clumps reported in the literature.The $200 \rm{pc}$ resolved [ $\ion{C}{2}$ ] /FIR ratio follows the global relation with the FIR surface brightness established in low- and high-redshift galaxies, even at the quasar location.We find that dust continuum is emitted in regions of $\sim0\farcs02-0\farcs04$ consistent with the size of photo-dissociation regions (PDR), whereas $50\%$ of the [ $\ion{C}{2}$ ] originates from larger physical scales ( $\theta \gtrsim 2"$ ). The large-scale [ $\ion{C}{2}$ ] emission presents a velocity gradient aligned with a nearby companion with perturbed kinematics, and misaligned with the kinematics of the small-scale emission. The absence of significant [ $\ion{C}{2}$ ] emission by structures with physical scale $\lesssim 1 \rm{kpc}$ implies that [ $\ion{C}{2}$ ] emission is not produced in dense PDR located at the boundary of Giant Molecular Clouds. We argue instead that [ $\ion{C}{2}$ ] is produced in low-density PDRs in the interstellar medium and diffuse $\ion{H}{1}$ gas tidally-stripped during the ongoing merger.

</div>

<div id="div_fig1">

<img src="tmp_2412.07474/figures/J0305_quasar_Fig1_NA_multi_C2357_2sigma_maskgrow_gaussian.png" alt="Fig9" width="100%"/>

**Figure 9. -** Top: FIR continuum at $\sim 260 \rm{GHz}$ and velocity-integrated [$\ion${C}{2}] emission of the high--redshift quasar J0305--3150, based on all available ALMA observations, including earlier ‘lower'--resolution data published by Venemans2016, Venemans2019. The contours start at $\pm2\sigma$ and increase in powers of two. Positive contours are shown in full dark lines, and negative ones in dashed grey. The synthesized beam is plotted in the bottom left corner of each plot. The GAIA-corrected optical position of the quasar is shown with a star \citep[][]{Venemans2019}. Bottom: Mean velocity and velocity dispersion map of the [$\ion${C}{2}] emission, computed using a Gaussian fit in pixels where [$\ion${C}{2}] is detected at $>2\sigma$. The mask for the moment 1 and 2 maps is given by the moment 0 $2\sigma$ contours, with five rounds of binary erosion and binary dilation to remove small structures due to noise (see text for more details). (*fig:fig1*)

</div>
<div id="div_fig2">

<img src="tmp_2412.07474/figures/J0305_NA_multi_C2357_continuum_SFRD_withlabels.png" alt="Fig3" width="100%"/>

**Figure 3. -** Star-formation rate density ($\Sigma_{\rm{SFR}}$) map of J0305--3150 assuming a proportional relation between the dust emission and the $\Sigma_{\rm{SFR}}$. The contours start at $\pm2\sigma$ and increase in steps of $2\sigma$(up to $8\sigma$ only). The synthesized beam is plotted in the bottom left corner. The gray box shows the area tessellated with independent $r=0$\farcs$037 (200 \rm{pc}$  apertures used to study the resolved FIR properties (see further text and Fig. \ref{fig:resolved_CII_deficit}. The cyan circles show $r=0$\farcs$037$ apertures corresponding to regions of interest (see further text and Table \ref{tab:clumps_properties}).  (*fig:sfrd_map*)

</div>
<div id="div_fig3">

<img src="tmp_2412.07474/figures/cdf_pdf_dust_cii_emission.png" alt="Fig10" width="100%"/>

**Figure 10. -** **Left panel:** Real part of the visibilities for the continuum and [$\ion${C}{2}] emission, averaged as a function of baseline length converted to physical scales. The errorbars represent the standard deviation in each bin, and the visibilities are normalised to the first datapoint (_uv_ distances $(15\pm1)\rm{m}$). The best-fit cumulative distribution functions are shown in dashed black. **Right panel:** Best-fit probability density function for the continuum and [$\ion${C}{2}] emission as a function of the logarithm of the physical scale observed. The maximum recoverable scale of the ALMA C10 observations is shown in dashed black. We also indicate the  photo-dissociation regions (PDR) sizes where [$\ion${C}{2}] is thought to originate found in simulations and observations at $0\lesssim z\lesssim 5$(black markers and errors), and provide a similar estimate for J0305--3150 in dark red.  \nocite{Carlstrom1991} (*fig:visiblities_scales*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2412.07474"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\AS}[3]{#1 ^{+#2}_{-#3}}$
$\newcommand{\ASmath}[3]{#1^{+#2}_{-#3}}$
$\newcommand{\serval}{\texttt{serval}}$
$\newcommand{\sim}{{\oldsim}}$
$\newcommand{\bfseries}{\fontseries{b}\selectfont}$</div>



<div id="title">

# The CARMENES search for exoplanets around M dwarfs: The impact of rotation and magnetic fields on the radial velocity jitter in cool stars

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2412.07691-b31b1b.svg)](https://arxiv.org/abs/2412.07691)<mark>Appeared on: 2024-12-11</mark> - 

</div>
<div id="authors">

H. L. Ruh, et al. -- incl., <mark>T. Henning</mark>

</div>
<div id="abstract">

**Abstract:** Radial velocity (RV) jitter represents an intrinsic limitation on the precision of Doppler searches for exoplanets that can originate from both instrumental and astrophysical sources. We aim to determine the RV jitter floor in M dwarfs and investigate the stellar properties that lead to RV jitter induced by stellar activity. We determined the RV jitter in $\num{239}$ M dwarfs from the CARMENES survey that are predominantly of mid to late spectral type and solar metallicity. We also investigated the correlation between stellar rotation and magnetic fields with RV jitter. The median jitter in the CARMENES sample is $\SI{3.1}{\meter\per\second}$ , and it is $\SI{2.3}{\meter\per\second}$ for stars with an upper limit of $\SI{2}{\kilo\meter\per\second}$ on their projected rotation velocities. We provide a relation between the stellar equatorial rotation velocity and RV jitter in M dwarfs based on a subsample of $\num{129}$ well-characterized CARMENES stars. RV jitter induced by stellar rotation dominates for stars with equatorial rotation velocities greater than $\SI{1}{\kilo\meter\per\second}$ . A jitter floor of $\SI{2}{\meter\per\second}$ dominates in stars with equatorial rotation velocities below $\SI{1}{\kilo\meter\per\second}$ . This jitter floor likely contains contributions from stellar jitter, instrumental jitter, and undetected companions. We study the impact of the average magnetic field and the distributions of magnetic filling factors on the RV jitter. We find a series of stars with excess RV jitter and distinctive distributions of magnetic filling factors. These stars are characterized by a dominant magnetic field component between $\SIrange{2}{4}{\kilo\gauss}$ . An RV jitter floor can be distinguished from RV jitter induced by activity and rotation based on the stellar equatorial rotation velocity. RV jitter induced by activity and rotation primarily depends on the equatorial rotation velocity. This RV jitter is also related to the distribution of magnetic filling factors, and this emphasizes the role of the magnetic field in the generation of RV jitter.

</div>

<div id="div_fig1">

<img src="tmp_2412.07691/./Fig/vrot_jitter_all_mcmc_variances_contributions.png" alt="Fig4" width="100%"/>

**Figure 4. -** Jitter-rotation relation for \num{129} CARMENES M dwarfs with known rotation periods. The RV jitter is fit as a function of the stellar rotation velocity $v_{\rm eq}$. The solid line displays the best fit, and the shaded region indicates the prediction interval. The jitter floor and the linear trend (dashed lines) correspond to parameters $\alpha$ and $\beta$ in Eq. (\ref{eq:jitter_vrot}). (*fig:jitter_vrot_all*)

</div>
<div id="div_fig2">

<img src="tmp_2412.07691/./Fig/jitter_bfield.png" alt="Fig6" width="100%"/>

**Figure 6. -** Radial velocity jitter vs. mean magnetic field. The upper limits on the average magnetic field are indicated by triangles. (*fig:jitter_bfield*)

</div>
<div id="div_fig3">

<img src="tmp_2412.07691/./Fig/vrot_jitter_bfield_structure.png" alt="Fig9" width="100%"/>

**Figure 9. -** As Fig. \ref{fig:jitter_vrot_all}, but color-coded with the filling factor distributions index and only for stars with measured average magnetic field. The labels mark the series of high-jitter stars with concentrated magnetic field distributions. The empty circles display stars without measurements of the average magnetic field. (*fig:bfield_structure*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2412.07691"></div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

278  publications files modified in the last 7 days.


In [11]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

15  publications in the last 7 days.


In [12]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [13]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [14]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

7  publications in the last day.


In [15]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
